In [173]:
import pandas as pd
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
import nltk
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
import gensim.downloader as api
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek
#fork

## Project 1 - NLP and Text Classification

For this project you will need to classify some angry comments into their respective category of angry. The process that you'll need to follow is (roughly):
<ol>
<li> Use NLP techniques to process the training data. 
<li> Train model(s) to predict which class(es) each comment is in.
    <ul>
    <li> A comment can belong to any number of classes, including none. 
    </ul>
<li> Generate predictions for each of the comments in the test data. 
<li> Write your test data predicitions to a CSV file, which will be scored. 
</ol>

You can use any models and NLP libraries you'd like. Think aobut the problem, look back to see if there's anything that might help, give it a try, and see if that helps. We've regularly said we have a "toolkit" of things that we can use, we generally don't know which ones we'll need, but here you have a pretty simple goal - if it makes it more accurate, it helps. There's not one specific solution here, there are lots of things that you could do. 

## Training Data

Use the training data to train your prediction model(s). Each of the classification output columns (toxic to the end) is a human label for the comment_text, assessing if it falls into that category of "rude". A comment may fall into any number of categories, or none at all. Membership in one output category is <b>independent</b> of membership in any of the other classes (think about this when you plan on how to make these predictions - it may also make it easier to split work amongst a team...). 

In [174]:
train_df = pd.read_csv("train.csv.zip")
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


#### Function Creation and Setup

In [175]:
toxic = train_df[["id","comment_text","toxic"]]

In [176]:
sToxic = train_df[["id","comment_text","severe_toxic"]]

In [177]:
obscene = train_df[["id","comment_text","obscene"]]

In [178]:
threat = train_df[["id","comment_text","threat"]]

In [179]:
idHate = train_df[["id","comment_text","identity_hate"]]

In [180]:
insult = train_df[["id","comment_text","insult"]]

#### Model Creation

In [181]:
def removeContractions(text):
    text = text.lower()
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\W",' ' , text)
    text = re.sub(r"\s+", ' ', text)
    text = text.strip(' ')
    return text

In [182]:
train_df["comment_text"] = train_df["comment_text"].apply(lambda x: removeContractions(x))

In [183]:
#twitterEmbedded = MeanEmbeddingVectorizer(w2vTwitter)
tfidfVectorizor = TfidfVectorizer(max_features=5000,stop_words='english')
cleanText = tfidfVectorizor.fit_transform(train_df['comment_text'])

In [184]:
gridParams = {'penalty':['l1','l2','elasticnet','none'],
              'dual':[True,False],
              'C':range(10,13),
              'solver':['liblinear'],
              'l1_ratio':np.arange(0,1,0.2)}

# Toxic

In [185]:
 # Split data - using the new dataframe parts that we cleaned up. 
#X_train, X_test, y_train, y_test = train_test_split(cleanText,toxic["toxic"])


#model fitting
#toxicModel = LogisticRegression()

#toxicGrid = GridSearchCV(estimator = toxicModel, param_grid=gridParams, cv=5)

#toxicGrid.fit(X_train, y_train)
toxicBest = LogisticRegression(C=10, dual=True, l1_ratio=0.0, solver='liblinear')
toxicBest.fit(cleanText, toxic['toxic'])

#Predict y value for test dataset
# y_predict = toxicBest.predict(X_test)
# y_prob = toxicBest.predict_proba(X_test)[:,1]
 
# #calculate scores
# fpr, tpr, thresholds = roc_curve(y_test, y_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(y_test, y_predict))
# print("Toxic best estimator:", toxicBest)
# sns.heatmap(confusion_matrix(y_test, y_predict), annot=True)

c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


LogisticRegression(C=10, dual=True, l1_ratio=0.0, solver='liblinear')

# Severely Toxic

In [186]:
 # Split data - using the new dataframe parts that we cleaned up. 
#stX_train, stX_test, sty_train, sty_test = train_test_split(cleanText,severeToxic["severe_toxic"])

#model fitting
# severeToxicModel = LogisticRegression(n_jobs=-1)
# sToxicGrid = GridSearchCV(estimator = severeToxicModel, param_grid= gridParams, cv=5)

# sToxicGrid.fit(stX_train, sty_train)
sToxicBest = LogisticRegression(C=10, dual=True, l1_ratio=0.0, n_jobs=-1, solver='liblinear')
sToxicBest.fit(cleanText, sToxic['severe_toxic'])
#Predict y value for test dataset
# sty_predict = sToxicBest.predict(stX_test)
# sty_prob = sToxicBest.predict_proba(stX_test)[:,1]
 
# #calculate scores
# fpr, tpr, thresholds = roc_curve(sty_test, sty_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(sty_test, sty_predict))
# sns.heatmap(confusion_matrix(sty_test, sty_predict), annot=True)

c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(


LogisticRegression(C=10, dual=True, l1_ratio=0.0, n_jobs=-1, solver='liblinear')

# Obscene

In [187]:
 # Split data - using the new dataframe parts that we cleaned up. 
#oX_train, oX_test, oy_train, oy_test = train_test_split(cleanText,obscene["obscene"])

#model fitting
#obsceneModel = LogisticRegression(n_jobs=-1)
#obsceneGrid = GridSearchCV(estimator=obsceneModel, param_grid=gridParams, cv=5)
#obsceneGrid.fit(oX_train, oy_train)
obsceneBest = LogisticRegression(C=10, dual=True, l1_ratio=0.0, solver='liblinear')
obsceneBest.fit(cleanText, obscene['obscene'])

#Predict y value for test dataset
# oy_predict = obsceneBest.predict(oX_test)
# oy_prob = obsceneBest.predict_proba(oX_test)[:,1]
 
# #calculate scores
# fpr, tpr, thresholds = roc_curve(oy_test, oy_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(oy_test, oy_predict))
# sns.heatmap(confusion_matrix(oy_test, oy_predict), annot=True)

c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


LogisticRegression(C=10, dual=True, l1_ratio=0.0, solver='liblinear')

# Insult

In [188]:
 # Split data - using the new dataframe parts that we cleaned up. 
#iX_train, iX_test, iy_train, iy_test = train_test_split(cleanText,insult["insult"])

#model fitting
# insultModel = LogisticRegression(n_jobs=-1)
# insultGrid = GridSearchCV(estimator=insultModel, param_grid=gridParams, cv= 5)
# insultGrid.fit(iX_train, iy_train)

insultBest = LogisticRegression(C=10, dual=True, l1_ratio=0.0, n_jobs=-1, solver='liblinear')
insultBest.fit(cleanText, insult['insult'])

#Predict y value for test dataset
# iy_predict = insultBest.predict(iX_test)
# iy_prob = insultBest.predict_proba(iX_test)[:,1]
 
# #calculate scores
# fpr, tpr, thresholds = roc_curve(iy_test, iy_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(iy_test, iy_predict))
# sns.heatmap(confusion_matrix(iy_test, iy_predict), annot=True)

c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(


LogisticRegression(C=10, dual=True, l1_ratio=0.0, n_jobs=-1, solver='liblinear')

# Identity Hate

In [189]:
 # Split data - using the new dataframe parts that we cleaned up. 
#idhX_train, idhX_test, idhy_train, idhy_test = train_test_split(cleanText,identityHate["identity_hate"])

#model fitting
# identityHateModel = LogisticRegression(n_jobs=-1)
# identityHateGrid = GridSearchCV(estimator=identityHateModel, param_grid=gridParams, cv=5)
# identityHateGrid.fit(idhX_train, idhy_train)
identityHateBest = LogisticRegression(C=10, dual=True, l1_ratio=0.0, n_jobs=-1, solver='liblinear')
identityHateBest.fit(cleanText, idHate['identity_hate'])

#Predict y value for test dataset
# idhy_predict = identityHateBest.predict(idhX_test)
# idhy_prob = identityHateBest.predict_proba(idhX_test)[:,1]
 
# #calculate scores
# fpr, tpr, thresholds = roc_curve(idhy_test, idhy_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(idhy_test, idhy_predict))
# sns.heatmap(confusion_matrix(idhy_test, idhy_predict), annot=True)

c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(


LogisticRegression(C=10, dual=True, l1_ratio=0.0, n_jobs=-1, solver='liblinear')

# Threat

In [190]:
 # Split data - using the new dataframe parts that we cleaned up. 
#tX_train, tX_test, ty_train, ty_test = train_test_split(cleanText,threat["threat"])

#model fitting
# threatModel = LogisticRegression(n_jobs=-1)
# threatGrid = GridSearchCV(estimator=threatModel, param_grid=gridParams, cv=5)
# threatGrid.fit(tX_train, ty_train)
threatBest = LogisticRegression(C=10, dual=True, l1_ratio=0.0, n_jobs=-1, solver='liblinear')
threatBest.fit(cleanText, threat['threat'])

#Predict y value for test dataset
# ty_predict = threatBest.predict(tX_test)
# ty_prob = threatBest.predict_proba(tX_test)[:,1]
 
# #calculate scores
# fpr, tpr, thresholds = roc_curve(ty_test, ty_prob)
# roc_auc = auc(fpr, tpr)
# print('AUC:', roc_auc)  

# print(classification_report(ty_test, ty_predict))
# sns.heatmap(confusion_matrix(ty_test,ty_predict), annot=True)

c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1106: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
c:\Users\atrsy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(


LogisticRegression(C=10, dual=True, l1_ratio=0.0, n_jobs=-1, solver='liblinear')

## Test Data

In [191]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,comment_text
0,1,Yo bitch Ja Rule is more succesful then you'll...
1,2,== From RfC == \n\n The title is fine as it is...
2,3,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,4,":If you have a look back at the source, the in..."
4,5,I don't anonymously edit articles at all.


In [197]:
test_df['comment_text'] = test_df['comment_text'].apply(lambda x: removeContractions(x))
testCleanText = tfidfVectorizor.fit_transform(test_df['comment_text'])

In [198]:
# Toxic final prediction
yToxic = toxicBest.predict(testCleanText)

#Severely toxic final prediction
ySevere = sToxicBest.predict(testCleanText)

#Obscene final prediction
yObscene = obsceneBest.predict(testCleanText)

#Insult final prediction
yInsult = insultBest.predict(testCleanText)

#Identity Hate final prediction
yIDHate = identityHateBest.predict(testCleanText)

#Threat final prediction
yThreat = threatBest.predict(testCleanText)

In [199]:
#now kith
columns = ["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
thisIsGoingOut = pd.DataFrame(list(zip(test_df["id"], yToxic, ySevere,yObscene, yThreat, yInsult, yIDHate)),
                              columns=columns)
thisIsGoingOut.to_csv('out.csv', index=False)  

## Output Details, Submission Info, and Example Submission

For this project, please output your predictions in a CSV file. The structure of the CSV file should match the structure of the example below. 

The output should contain one row for each row of test data, complete with the columns for ID and each classification.

Into Moodle please submit:
<ul>
<li> Your notebook file(s). I'm not going to run them, just look. 
<li> Your sample submission CSV. This will be evaluated for accuracy against the real labels; only a subset of the predictions will be scored. 
</ul>

It is REALLY, REALLY, REALLY important the the structure of your output matches the specifications. The accuracies will be calculated by a script, and it is expecting a specific format. 

### Sample Evaluator

The file prediction_evaluator.ipynb contains an example scoring function, scoreChecker. This function takes a sumbission and an answer key, loops through, and evaluates the accuracy. You can use this to verify the format of your submission. I'm going to use the same function to evaluate the accuracy of your submission, against the answer key (unless I made some mistake in this counting function).

In [200]:
#Construct dummy data for a sample output. 
#You won't do this part first, you have real data - I'm faking it. 
#Your data should have the same structure, so the CSV output is the same
# dummy_ids = ["dfasdf234", "asdfgw43r52", "asdgtawe4", "wqtr215432"]
# dummy_toxic = [0,0,0,0]
# dummy_severe = [0,0,0,0]
# dummy_obscene = [0,1,1,0]
# dummy_threat = [0,1,0,1]
# dummy_insult = [0,0,1,0]
# dummy_ident = [0,1,1,0]
# columns = ["id", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
# sample_out = pd.DataFrame( list(zip(dummy_ids, dummy_toxic, dummy_severe, dummy_obscene, dummy_threat, dummy_insult, dummy_ident)),
#                     columns=columns)
# sample_out.head()

In [201]:
#Write DF to CSV. Please keep the "out.csv" filename. Moodle will auto-preface it with an identifier when I download it. 
#This command should work with your dataframe of predictions. 
# sample_out.to_csv('out.csv', index=False)  

## Grading

The grading for this is split between accuracy and well written code:
<ul>
<li> 75% - Accuracy. The most accurate will get 100% on this, the others will be scaled down from there. 
<li> 25% - Code quality. Can the code be followed and made sense of - i.e. comments, sections, titles. 
</ul>